In [95]:
import numpy as np

input_data = np.array([[1, 2, 3, 0, 5], [0, 1, 2, 3, 4], [3, 0, 1, 2, 7], [2, 3, 0, 1, 2], [2, 3, 4, 5, 6]])
filter = np.array([[2, 0, 1], [0, 1, 2], [1, 0, 2]])
stride = 1

loop = int((input_data.shape[0] - filter.shape[0]) / stride + 1)
answer = np.zeros((loop, loop))

for i in range(loop): 
    for j in range(loop):
        answer[i][j] = np.sum(input_data[i:i+filter.shape[0], j:j+filter.shape[0]] * filter)

print(answer)

[[15. 16. 37.]
 [ 6. 15. 28.]
 [20. 17. 30.]]


In [2]:
import numpy as np
x = np.random.rand(10, 1, 28, 28) # 높이 28, 너비 28, 채널 1인 데이터가 10개.
x.shape

(10, 1, 28, 28)

In [4]:
x[0].shape
x[1].shape

(1, 28, 28)

In [6]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        FN, C, FH, FW = self.W.shape # 필터 개수, 채널, 필터 높이, 필터 너비
        N, C, H, W = x.shape # 데이터 수, 채널, 높이, 너비

        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # self.W는 (10, 3, 5, 5) 형태. self.W의 배열의 원소 수는 750개. reshape(FN(10), -1)을 하면 750개 원소를 FN(10)묶음으로 만든다. -> (10, 75)의 형태. -> Transpose 해서 (75, 10) 꼴.
        out = np.dot(col, col_W) + self.b # 행렬 계산.

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2) # out은 (N, H, W, C) 꼴. transpose를 해서 (N, C, H, W) 꼴로 변경.

        return out

In [ ]:
class Pooling:
    def __init__ (self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h, self.pool_w)

        out = np.max(col, axis=1)

        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

## 7.5 CNN 구현하기

In [10]:
import sys, os
sys.path.append(os.pardir)
from collections import OrderedDict
from common.layers import Relu, Affine, SoftmaxWithLoss, Convolution

class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param["filter_num"]
        filter_size = conv_param["filter_size"]
        filter_pad = conv_param["pad"]
        filter_stride = conv_param["stride"]
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size / 2) * (conv_output_size / 2))

        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params["b1"] = np.zeros(filter_size)
        self.params["W2"] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params["b2"] = np.zeros(hidden_size)
        self.params["W3"] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params["b3"] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers["Conv1"] = Convolution(self.params["W1"], self.params["b1"], conv_param["stride"], conv_param["pad"])
        self.layers["Relu1"] = Relu()
        self.layers["Pool1"] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers["Affine1"] = Affine(self.params["W2"], self.params["b2"])
        self.layers["Relu2"] = Relu()
        self.layers["Affine2"] = Affine(self.params["W3"], self.params["b3"])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(x, t)
    
    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers)
        layers.reverse()

        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads["W1"] = self.layers["Conv1"].dW
        grads["b1"] = self.layers["Conv1"].db
        grads["W2"] = self.layers["Affine1"].dW
        grads["b2"] = self.layers["Affine1"].db
        grads["W3"] = self.layers["Affine2"].dW
        grads["b3"] = self.layers["Affine2"].db

        return grads